In [2]:
import torch
import torch.nn as nn
import mmap
import random
from torch.nn import functional as F

# check if Metal MPS available
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    mps_device = "cpu"
    print ("MPS device not found.")

# set necessary parameters
block_size = 64
batch_size = 128
max_iters = 1000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

tensor([1.], device='mps:0')


In [2]:
# Read the text file
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Get unique characters in the text
chars = sorted(set(text))

# Calculate the vocabulary size
vocab_size = len(chars)

In [3]:
# encode / decode stuff - TOKENIZER
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [4]:
# This function gets a random chunk of data for a given split
def get_random_chunk(split):
    # Select the appropriate file based on the split
    filename = 'train_split.txt' if split == 'train' else 'val_split.txt'
    
    # Open the file in read-only binary mode
    with open(filename, 'rb') as f:
        # Create a memory-mapped file object
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Get the size of the file
            file_size = len(mm)
            
            # Choose a random starting position within the file
            start_pos = random.randint(0, file_size - block_size*batch_size)

            # Seek to the starting position
            mm.seek(start_pos)
            
            # Read the block of data
            block = mm.read(block_size*batch_size-1)

            # Decode the block of data and remove any carriage returns
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Encode the decoded block as a tensor of integers
            data = torch.tensor(encode(decoded_block), dtype=torch.long)

    return data        


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(mps_device), y.to(mps_device)
    return x,y

In [5]:
@torch.no_grad()
def estimate_loss():
    """
    Estimates the loss for the model on the train and validation splits.

    Returns:
    - out (dict): A dictionary containing the mean loss for each split.
    """
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
# Define the Head class for the multi-head attention mechanism
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)  # Linear transformation for keys
        self.query = nn.Linear(n_embd, head_size, bias=False)  # Linear transformation for queries
        self.value = nn.Linear(n_embd, head_size, bias=False)  # Linear transformation for values
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))  # Lower triangular mask
        self.dropout = nn.Dropout(dropout)  # Dropout layer

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)  # Compute keys
        q = self.query(x)  # Compute queries

        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5  # Compute attention weights
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # Apply lower triangular mask
        wei = F.softmax(wei, dim=-1)  # Apply softmax to get attention probabilities
        wei = self.dropout(wei)  # Apply dropout

        v = self.value(x)  # Compute values
        out = wei @ v  # Weighted sum of values
        return out


# Define the MultiHeadAttention class
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])  # List of head modules
        self.proj = nn.Linear(head_size * num_heads, n_embd)  # Linear transformation for projection
        self.dropout = nn.Dropout(dropout)  # Dropout layer

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # Concatenate outputs from all heads
        out = self.dropout(self.proj(out))  # Apply dropout and projection
        return out


# Define the FeedForward class
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),  # Linear transformation
            nn.ReLU(),  # ReLU activation
            nn.Linear(4*n_embd, n_embd),  # Linear transformation
            nn.Dropout(dropout),  # Dropout layer
        )
        
    def forward(self, x):
        return self.net(x)  # Pass input through the network


# Define the Block class
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)  # Self-attention mechanism
        self.ffwd = FeedForward(n_embd)  # Feed-forward network
        self.ln1 = nn.LayerNorm(n_embd)  # Layer normalization
        self.ln2 = nn.LayerNorm(n_embd)  # Layer normalization

    def forward(self, x):
        y = self.sa(x)  # Apply self-attention mechanism
        x = self.ln1(x+y)  # Residual connection and layer normalization
        y = self.ffwd(x)  # Apply feed-forward network
        x = self.ln2(x+y)  # Residual connection and layer normalization
        return x        


# Define the GPTLanguageModel class
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)  # Token embedding table
        self.position_embedding_table = nn.Embedding(block_size, n_embd)  # Position embedding table
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])  # Sequence of blocks
        self.ln_f = nn.LayerNorm(n_embd)  # Layer normalization
        self.lm_head = nn.Linear(n_embd, vocab_size)  # Linear transformation for language modeling head

        self.apply(self._init_weights)  # Initialize weights

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)  # Initialize linear layer weights
            if module.bias is not None:
                nn.init.zeros_(module.bias)  # Initialize linear layer bias
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)  # Initialize embedding layer weights

    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index)  # Token embeddings
        pos_emb = self.position_embedding_table(torch.arange(T, device=mps_device))  # Position embeddings
        x = tok_emb + pos_emb  # Add token and position embeddings
        x = self.blocks(x)  # Pass through blocks
        x = self.ln_f(x)  # Layer normalization
        logits = self.lm_head(x)  # Compute logits
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)  # Compute cross-entropy loss
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]  # Get logits for the last token
            probs = F.softmax(logits, dim=-1)  # Compute probabilities
            index_next = torch.multinomial(probs, num_samples=1)  # Sample next token
            index = torch.cat((index, index_next), dim=1)  # Append next token to input sequence
        return index


# Create an instance of GPTLanguageModel
model = GPTLanguageModel(vocab_size)
m = model.to(mps_device)  # Move the model to the specified device

In [7]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# train the model
for iter in range(max_iters+1):
    if iter % eval_iters == 0:
        # Estimate the loss for the train and validation splits
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # Sample a batch
    xb, yb = get_batch('train')

    # Evaluate loss
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    # training step
    loss.backward()
    optimizer.step()

step 0: train loss 4.5171, val loss 4.5225
step 100: train loss 2.3555, val loss 2.4264
step 200: train loss 1.9606, val loss 2.0688
step 300: train loss 1.7219, val loss 1.8512
step 400: train loss 1.6038, val loss 1.7594
step 500: train loss 1.5127, val loss 1.6901
step 600: train loss 1.4381, val loss 1.6423
step 700: train loss 1.3826, val loss 1.6086
step 800: train loss 1.3379, val loss 1.5711
step 900: train loss 1.2989, val loss 1.5473
step 1000: train loss 1.2631, val loss 1.5381
1.3619656562805176


In [8]:
context = torch.zeros((1,1), dtype=torch.long, device=mps_device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

RuntimeError: The size of tensor a (64) must match the size of tensor b (65) at non-singleton dimension 2